In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import haiku as hk

In [2]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
from jax import jacfwd, jacrev
import numpy as np
from functools import partial
from jax.ops import index, index_add, index_update
import optax
import logging

In [3]:
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

In [22]:
# class MLP(hk.Module):
#     def __init__(self,c,group,ch=384,num_layers=3):
#         super().__init__()
#         self.chs = num_layers*[c]
#         logging.warning("Initing MLP")

#     def __call__(self,x,is_training):
#         for c in self.chs:
#             x = hk.Linear(c)(x)
#             x = hk.BatchNorm(True,True,.9)(x,is_training=is_training)
#             x = jax.nn.swish(x)
#         x = hk.Linear(1)(x)
#         return x.squeeze(-1)

# def net_fn(x,is_training):
#     return MLP(10,None)(x,is_training)

def get_mlp(ch=30,num_layers=3):
    logging.warning("Initing MLP")
    W = hk.get_parameter("w", shape=[5,4], init=jnp.zeros)
    def forward(x, is_training):
        for c in num_layers*[ch]:
            x = hk.Linear(c)(x)
            x = hk.BatchNorm(True,True,.9)(x,is_training=is_training)
            x = jax.nn.swish(x)
        x = hk.Linear(1)(x)
        return x.squeeze(-1)
    return forward

In [23]:
key = random.PRNGKey(0)
#net = hk.transform_with_state(lambda x,is_training: get_mlp(30,3)(x,is_training))
net = hk.transform_with_state(get_mlp(30,3))
x = random.normal(key,(30,10))
params,state = net.init(jax.random.PRNGKey(42), x,is_training=True)
bs=5000
opt = optax.adam(1e-3)
opt_state = opt.init(params)

ValueError: `hk.get_parameter` must be used as part of an `hk.transform`

In [21]:
for i in range(20):
    key = random.PRNGKey(i)
    x = random.normal(key,(30,10))
    np.asarray(net.apply(params,state,None,x,is_training=True))

In [24]:
jax.device_count()

1